In [1]:
import pandas as pd
df = pd.read_csv('df practice - シート1.csv')
df

,Unnamed: 0,2018,2019,2020,2021,2022
0,a,1,1,1,1,1
1,b,0,0,1,0,0
2,c,0,1,0,0,1
3,d,0,0,0,0,1
4,e,0,0,0,1,0


In [2]:
df=df.drop(['Unnamed: 0'],axis=1)

In [3]:
df.idxmax(axis=1)

0    2018
1    2020
2    2019
3    2022
4    2021
dtype: object

In [5]:
#death ダミー変数化
import numpy as np
import pandas as pd
import statsmodels.api as sm

#データのインポート
df = pd.read_csv("http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.csv")
df.head()

df = df.iloc[:,1:]
df1 = df[['cat1','sex','death','age','swang1','meanbp1','aps1']]
df1.head()

#傾向スコアを求めるのに使う説明変数
X = df1.drop(['death','swang1'],1)
dummy_X = pd.get_dummies(X[['cat1','sex']],drop_first=True)
X2 = pd.merge(X, dummy_X, left_index=True, right_index=True)
X = X2.drop(['cat1','sex'],1)

#施術したかどうかダミー変数化
ps_target = df1['swang1']
dummy_ps = pd.get_dummies(ps_target)
ps_target = dummy_ps['RHC']

#傾向スコアを求める
glm = sm.Logit(ps_target, X)
result = glm.fit()
ps = pd.Series(result.predict(X))

y = df1['death']
dummy_y = pd.get_dummies(y)
y = dummy_y['Yes']

ps = pd.Series(result.predict(X))
z1 = ps_target

#Propensity score matching
table = pd.concat([ps,z1,y],axis=1)
table.columns = ['ps','treat','died']

interval = np.arange(0,1.05,0.05)
match_list = []
for i in range(0,len(interval)-1):
    temp0 = table[(table['treat']==0) & (interval[i] < table['ps']) & (table['ps'] < interval[i+1])]
    temp1 = table[(table['treat']==1) & (interval[i] < table['ps']) & (table['ps'] < interval[i+1])]
    if (len(temp0) > 0) & (len(temp1) > 0):
        match_list.append(temp1['died'].mean()-temp0['died'].mean())

np.mean(match_list)

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>